In [1]:
import os, json, re
import random, time
import pandas as pd, numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
from dotenv import load_dotenv

from itertools import islice

# Util built for sharktank project
import sharktank_utils_mod as su
from sharktank_utils_mod import load_facts, PitchOrchestrator, metrics_calculation, PitchEditor, OrchestratorResponse, PitchEquipped

# Agno for llm agents
from agno.agent import Agent
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.wikipedia import WikipediaTools

# Other LLM utils
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
reference_models = su.REFERENCE_MODELS
agent_storage = su.agent_storage
edit_reference_models = su.EDIT_REFERENCE_MODELS

In [4]:
facts_store = load_facts()
pitches = {k:"" for k,_ in facts_store.items()}
metrics = {k:"" for k,_ in facts_store.items()}
time_stamps = {k:"" for k,_ in facts_store.items()}

In [5]:
# Main loop
orcheditor = PitchEquipped(reference=edit_reference_models, iterations=2)
goal = 'create a pitch with the given facts'

RUN_FIRST_N = True      # Change to False to run all
N = 2

cases_to_run = islice(facts_store.items(), N) if RUN_FIRST_N else facts_store.items()
for case, facts in tqdm(cases_to_run, desc="Generating pitches"):
    # case = 'facts_shark_tank_transcript_37_TIK PIK.txt'
    lean_facts = {
        'facts': facts['facts'],
        'product_description': facts['product_description']
    }
    try:
        pitch = orcheditor.orchestrate_with_edit(goal, lean_facts, verbose=False)
        pitches[case] = pitch
        metrics[case] = orcheditor.logs
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        print(f"{case} error: {e}")
        pitches[case] = ""
        metrics[case] = []
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Reset for next run
    orcheditor.logs = []
    orcheditor.agents = {}

Generating pitches: 0it [00:00, ?it/s]

INFO     Searching wikipedia for: customer acquisition and retention strategies

WARNING  Could not run function search_wikipedia(query=customer acquisition and retention strategies)

ERROR    Page id "customer acquisition and reception strategic" does not match any pages. Try another id!          
         Traceback (most recent call last):                                                                        
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\agno\tools\function.py", line 363, in   
         execute                                                                                                   
             self.result = self.function.entrypoint(**entrypoint_args, **self.arguments)                           
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\pydantic\_internal\_validate_call.py",  
         line 38, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\pydantic\_internal\_validate_call.py",  
         line 111, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\agno\tools\wikipedia.py", line 54, in   
         search_wikipedia                                                                                          
             return json.dumps(Document(name=query, content=wikipedia.summary(query)).to_dict())                   
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\wikipedia\util.py", line 28, in __call__
             ret = self._cache[key] = self.fn(*args, **kwargs)                                                     
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\wikipedia\wikipedia.py", line 231, in   
         summary                                                                                                   
             page_info = page(title, auto_suggest=auto_suggest, redirect=redirect)                                 
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\wikipedia\wikipedia.py", line 276, in   
         page                                                                                                      
             return WikipediaPage(title, redirect=redirect, preload=preload)                                       
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\wikipedia\wikipedia.py", line 299, in   
         __init__                                                                                                  
             self.__load(redirect=redirect, preload=preload)                                                       
           File "c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\wikipedia\wikipedia.py", line 345, in   
         __load                                                                                                    
             raise PageError(self.title)                                                                           
         wikipedia.exceptions.PageError: Page id "customer acquisition and reception strategic" does not match any 
         pages. Try another id!

INFO     Searching wikipedia for: customer retention strategies

[00:45<00:00] 2/2 | Running Pitch Editing Iterations: 100%|██████████ 22.52s/it
Generating pitches: 1it [00:45, 45.04s/it]

INFO     Searching wikipedia for: electric vehicle range anxiety

[00:48<00:00] 2/2 | Running Pitch Editing Iterations: 100%|██████████ 24.22s/it
Generating pitches: 2it [01:33, 46.74s/it]


In [6]:
calculated_metrics = []
for product, metric in metrics.items():
    calculated_metrics.append(metrics_calculation(metric))
input_lengths, output_lengths, latencies = zip(*calculated_metrics)

In [8]:
print(pitches['facts_shark_tank_transcript_0_GarmaGuard.txt'])
print(pitches['facts_shark_tank_transcript_0_Roadie.txt'])

{
    "Pitch": "Introducing GarmaGuard, a revolutionary natural garment and fabric cleanser with a strong competitive advantage. With an existing loyal customer base of 14,000, we are poised for expansion through a strategic customer acquisition and retention strategy. By exceeding customer expectations, implementing a loyalty program, leveraging influencer marketing, content marketing, referral programs, social media engagement, and email marketing, we aim to increase customer retention rates and attract new customers. Our marketing and sales plans are focused on building strong relationships and providing value beyond the product itself, propelling GarmaGuard to the forefront of the eco-friendly and natural living markets.",
    "Initial_Offer": {
        "Valuation": "$10 million",
        "Equity_Offered": "10%",
        "Funding_Amount": "$1 million",
        "Key_Terms": "Investment will be used to drive customer acquisition and retention strategies, further develop the product, 

Show empty ones

In [ ]:
for k, v in pitches.items():
    if v == "":
        print(k)

In [49]:
timestamp = datetime.now().strftime('%Y-%d-%m')
framework = "orchestrator"
layer = "orchestrator_1"
pitches_df = pd.DataFrame(
    data={
        'scenario_name':list(pitches.keys()),
        'framework':len(pitches) * [framework],
        'layer':len(pitches) * [layer],
        'model_name': len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'model_identity':len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'latency':latencies,
        'input_length':input_lengths,
        'output_length':output_lengths,
        'time_stamp':list(time_stamps.values()),
        'response':list(pitches.values()),
    }
)


In [ ]:
pitches_df

In [ ]:
pitches_df.to_excel(f"{timestamp}-orchestrator-editRAG-pitches.xlsx")